In [1]:
output_path = r'D:\Github\LLaVA-GI\data\validation\output_results5minigpt.json'
# But this is input file, not output file!!!

# ROUGE-1 Recall and ROUGE-1 Precision

In [2]:
import json
from collections import Counter

# 文件路径


def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except json.JSONDecodeError as e:
        print(f"Error reading JSON file {file_path}: {e}")
        return None

def calculate_rouge_1(human_answer, machine_answer):
    human_tokens = human_answer.split()
    machine_tokens = machine_answer.split()
    
    human_counter = Counter(human_tokens)
    machine_counter = Counter(machine_tokens)
    
    overlap = sum((human_counter & machine_counter).values())
    
    recall = overlap / len(human_tokens) if len(human_tokens) > 0 else 0
    precision = overlap / len(machine_tokens) if len(machine_tokens) > 0 else 0
    
    return recall, precision

# 读取输出结果文件
output_data = read_json_file(output_path)
if output_data is None:
    raise ValueError(f"Failed to read output results file: {output_path}")

# 计算 ROUGE-1 Recall 和 Precision
total_recall = 0
total_precision = 0
count = 0

for item in output_data:
    human_answer = item["human_answer"]
    machine_answer = item["machine_answer"]
    
    recall, precision = calculate_rouge_1(human_answer, machine_answer)
    
    total_recall += recall
    total_precision += precision
    count += 1

average_recall = total_recall / count if count > 0 else 0
average_precision = total_precision / count if count > 0 else 0

print(f"ROUGE-1 Recall: {average_recall:.4f}")
print(f"ROUGE-1 Precision: {average_precision:.4f}")

ROUGE-1 Recall: 0.0588
ROUGE-1 Precision: 0.0022


# CIDEr (Consensus-based Image Description Evaluation)

You need to download cider from https://github.com/tylin/coco-caption

I made some change to let the cider_scorer work again (fit python3): chang all xrange to range, and change all iteritems() to items()

In [3]:
import sys
import json
from pathlib import Path

# 添加模块路径
cider_path = Path(r'D:\Github\LLaVA-GI\cider')
if cider_path not in sys.path:
    sys.path.append(str(cider_path))

# 导入 Cider 类
from cider import Cider

# 文件路径
output_path = r'D:\Github\LLaVA-GI\data\validation\output_results5minigpt.json'

def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except json.JSONDecodeError as e:
        print(f"Error reading JSON file {file_path}: {e}")
        return None

# 读取输出结果文件
output_data = read_json_file(output_path)
if output_data is None:
    raise ValueError(f"Failed to read output results file: {output_path}")

# 准备数据
gts = {}
res = {}
for i, item in enumerate(output_data):
    gts[i] = [item["human_answer"]]
    res[i] = [item["machine_answer"]]

# 计算 CIDEr 分数
cider_scorer = Cider()
score, scores = cider_scorer.compute_score(gts, res)

print(f"CIDEr Score: {score:.4f}")

CIDEr Score: 0.0001
